In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import os
import pandas as pd
from torchvision.io import read_image
from sklearn.model_selection import train_test_split
from lib.util import *
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

/Users/neiljanwani/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ik_df = pd.read_csv(r'AB06/level_imu_01_01.csv').astype('float').set_index('Header')
labels_df = pd.read_csv(r'AB06/level_gon_01_01.csv').astype('float').set_index('Header') / 100
labels_df

,HeelStrike,ToeOff
Header,,
14.420,0.031873,0.143900
14.425,0.035857,0.144943
14.430,0.039841,0.145985
14.435,0.043825,0.147028
14.440,0.047809,0.148071
...,...,...
29.735,0.992172,0.993300
29.740,0.994129,0.994975
29.745,0.996086,0.996650


In [4]:
ik_df = ik_df[15:29]
labels_df = labels_df[15:29]
df = labels_df.drop('ToeOff', axis=1)
df

,HeelStrike
Header,
15.000,0.494024
15.005,0.498008
15.010,0.501992
15.015,0.505976
15.020,0.509960
...,...
28.980,0.696673
28.985,0.698630
28.990,0.700587


In [5]:
pio.templates.default = "plotly_white"

plot_template = dict(
    layout=go.Layout({
        "font_size": 18,
        "xaxis_title_font_size": 24,
        "yaxis_title_font_size": 24})
)

fig = px.line(df, labels=dict(created_at="Date", value='Percent gait', Header='Time (s)'))
fig.show()

In [6]:
pred = []

N = 3
running = np.ones(N) * 2 # inital rough guess
start = df.index[0]
stepped = False
step = 0
for time, percent in zip(df.index, df['HeelStrike']):
    if percent <= 0.01:
        start = time
        stepped = False
    elif percent >= 0.99 and not stepped:
        running[step % N] = time - start
        step += 1
        stepped = True

    x = (time - start) / running.mean()
    pred.append(x) if x < 1. else pred.append(1.)

df['Model'] = pred

In [51]:
pio.templates.default = "plotly_white"

plot_template = dict(
    layout=go.Layout({
        "font_size": 18,
        "xaxis_title_font_size": 24,
        "yaxis_title_font_size": 24})
)

fig = px.line(df, labels=dict(created_at="Date", value='Percent gait', Header='Time (s)'))
fig.show()

In [52]:
accuracies = np.linspace(0, 0.25, num=26)
prediction_accs = [len([None for o, t in zip(df[:]['HeelStrike'], df[:]['Model']) if min(1 - abs(o - t), abs(o - t)) <= close]) / len(df[:]) for close in accuracies]

In [53]:
for cat, acc in zip(accuracies, prediction_accs):
    print(cat, acc)

0.0 0.005355230274901821
0.01 0.18921813637986434
0.02 0.25740806854694753
0.03 0.3156015708675473
0.04 0.35308818279186005
0.05 0.379507318814709
0.06 0.40271331667261695
0.07 0.42520528382720457
0.08 0.4476972509817922
0.09 0.4623348803998572
0.1 0.4759014637629418
0.11 0.4905390931810068
0.12 0.5044626918957515
0.13 0.5187433059621563
0.14 0.5290967511602999
0.15 0.5394501963584434
0.16 0.5490896108532667
0.17 0.5594430560514102
0.18 0.5687254551945734
0.19 0.5787218850410568
0.2 0.58836129953588
0.21 0.5983577293823634
0.22 0.6079971438771867
0.23 0.61763655837201
0.24 0.6272759728668332
0.25 0.6372724027133166
